In [ ]:
#!/usr/bin/env python3
"""
Shared Infrastructure
=====================
Data loading, feature engineering, and Markov-switching regime detection.
Run this cell first; all subsequent cells depend on the objects created here.

Data: 8 monthly US economic series from FRED (1954-2025), merged into a
single panel with 833 observations after computing derived variables.

Key methodological choices:
  - CBO NAIRU (NROU, quarterly, interpolated monthly) for unemployment gap
  - Michigan Survey expected inflation (MICH, from 1978); adaptive proxy before
  - Hamilton (1989) 2-state Markov-switching model for regime identification
"""

import os, warnings, random
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import statsmodels.api as sm
from scipy.optimize import least_squares
from scipy import stats
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, roc_auc_score

warnings.filterwarnings("ignore")
SEED = 42; random.seed(SEED); np.random.seed(SEED)

DATA_PATH  = '/Users/leoss/Desktop/Portfolio/Website-/Central bank/data'
OUTPUT_PATH = '/Users/leoss/Desktop/Portfolio/Website-/Central bank/Outputs'
os.makedirs(OUTPUT_PATH, exist_ok=True)

C1, C2, C3, C4, C5 = "#2563eb", "#dc2626", "#7c3aed", "#f59e0b", "#10b981"
C_BG = "#fafafa"
plt.rcParams.update({
    "figure.facecolor": C_BG, "axes.facecolor": C_BG,
    "axes.grid": True, "grid.color": "#e5e7eb", "grid.linewidth": 0.5,
    "font.size": 11, "axes.spines.top": False, "axes.spines.right": False,
})

RECESSIONS = [
    ("1973-11","1975-03"),("1980-01","1980-07"),("1981-07","1982-11"),
    ("1990-07","1991-03"),("2001-03","2001-11"),("2007-12","2009-06"),("2020-02","2020-04"),
]
FED_CHAIRS = [
    ("Burns","1970-02","1978-01"),("Miller","1978-03","1979-08"),
    ("Volcker","1979-08","1987-08"),("Greenspan","1987-08","2006-01"),
    ("Bernanke","2006-02","2014-01"),("Yellen","2014-02","2018-02"),("Powell","2018-02","2024-12"),
]
CHAIR_DATES  = [(ch, pd.Timestamp(s), pd.Timestamp(e)) for ch, s, e in FED_CHAIRS]
CHAIR_COLORS = {"Burns":"#dbeafe","Miller":"#fee2e2","Volcker":"#ede9fe",
                "Greenspan":"#fef3c7","Bernanke":"#d1fae5","Yellen":"#fce7f3","Powell":"#e0e7ff"}
REGIME_COLORS = [C5, C2]

def shade_recessions(ax):
    for s, e in RECESSIONS:
        ax.axvspan(pd.Timestamp(s), pd.Timestamp(e), alpha=0.10, color="#fca5a5", zorder=0)

def load_data(path=DATA_PATH):
    def _read(names):
        for n in names:
            p = os.path.join(path, n)
            if os.path.exists(p):
                return pd.read_csv(p, parse_dates=["observation_date"], index_col="observation_date")
        raise FileNotFoundError(f"None of {names} in {path}")
    cpi    = _read(["CPIAUCSL.csv"])
    unrate = _read(["UNRATE.csv"])
    ff     = _read(["FEDFUNDS.csv", "FEDFUNDS-1.csv"])
    tcu    = _read(["TCU.csv"])
    gs10   = _read(["GS10.csv"])
    nfci   = _read(["NFCI.csv"]).resample("MS").last()
    nrou_q = _read(["NROU.csv", "NROUST.csv"])
    nrou_q.index = pd.DatetimeIndex(nrou_q.index)
    nrou_m = nrou_q.resample("MS").interpolate("linear")
    try: mich = _read(["MICH.csv"])
    except FileNotFoundError: mich = None
    merged = pd.concat([
        cpi.rename(columns={cpi.columns[0]: "cpi"}),
        unrate.rename(columns={unrate.columns[0]: "unemployment"}),
        ff.rename(columns={ff.columns[0]: "fed_funds"}),
        tcu.rename(columns={tcu.columns[0]: "capacity_util"}),
        gs10.rename(columns={gs10.columns[0]: "treasury_10y"}),
        nfci.rename(columns={nfci.columns[0]: "fin_conditions"}),
        nrou_m.rename(columns={nrou_m.columns[0]: "nrou"}),
    ], axis=1).dropna(subset=["cpi","unemployment","fed_funds","nrou"])
    if mich is not None:
        merged = merged.join(mich.rename(columns={mich.columns[0]: "expected_inflation"}), how="left")
    else: merged["expected_inflation"] = np.nan
    return merged

def engineer_features(data):
    df = data.copy()
    df["inflation"]     = df["cpi"].pct_change(12) * 100
    df["unemp_gap"]     = df["unemployment"] - df["nrou"]
    df["capacity_gap"]  = df["capacity_util"] - 100.0
    df["term_spread"]   = df["treasury_10y"] - df["fed_funds"]
    df["real_rate"]     = df["fed_funds"] - df["inflation"]
    df["delta_ff"]      = df["fed_funds"].diff()
    df["delta_pi"]      = df["inflation"].diff()
    df["delta_u"]       = df["unemployment"].diff()
    df["ff_lag1"]       = df["fed_funds"].shift(1)
    df["adaptive_pi_e"] = df["inflation"].rolling(12).mean()
    df["pi_expected"]   = df["expected_inflation"].fillna(df["adaptive_pi_e"])
    for var in ["inflation","unemp_gap","capacity_gap","fed_funds","term_spread","fin_conditions"]:
        for lag in [1, 3, 6, 12]:
            df[f"L{lag}_{var}"] = df[var].shift(lag)
    for h in [1, 3, 6, 12]:
        df[f"inflation_{h}m_ahead"] = df["inflation"].shift(-h)
    df["recession"] = 0
    for start, end in RECESSIONS:
        df.loc[(df.index >= start) & (df.index <= end), "recession"] = 1
    df = df.dropna(subset=["inflation","L12_inflation"])
    df.index = pd.DatetimeIndex(df.index)
    return df

def fit_regimes(df, n_regimes=2):
    endog = df["inflation"].copy()
    exog  = sm.add_constant(df[["unemployment"]].copy())
    mod   = sm.tsa.MarkovRegression(endog, k_regimes=n_regimes, exog=exog,
                switching_variance=True, switching_exog=True)
    best_res, best_llf = None, -np.inf
    for attempt in range(8):
        try:
            res = mod.fit(search_reps=30, random_state=SEED+attempt, disp=False, maxiter=500)
            if res.llf > best_llf: best_llf, best_res = res.llf, res
        except Exception: continue
    if best_res is None: raise RuntimeError("MS model failed")
    res = best_res; smoothed = res.smoothed_marginal_probabilities
    regime_means = {}
    for r in range(n_regimes):
        mask = smoothed[r] > 0.5
        regime_means[r] = df.loc[mask, "inflation"].mean() if mask.sum() > 0 else 0.0
    sorted_regimes = sorted(regime_means, key=regime_means.get)
    rdf = df.copy()
    for new_r in range(n_regimes):
        rdf[f"p_regime_{new_r}"] = smoothed[sorted_regimes[new_r]].values
    rdf["regime"] = np.argmax(
        np.column_stack([rdf[f"p_regime_{r}"].values for r in range(n_regimes)]), axis=1)
    sorted_means = [regime_means[sorted_regimes[r]] for r in range(n_regimes)]
    labels = ["Low","High"] if n_regimes == 2 else [f"R{r}" for r in range(n_regimes)]
    regime_names = {r: f"{labels[r]} inflation ({sorted_means[r]:.1f}%)" for r in range(n_regimes)}
    tm = res.regime_transition
    tm_avg = tm.mean(axis=2) if tm.ndim == 3 else tm
    return rdf, regime_names, n_regimes, {
        "result": res, "sorted_regimes": sorted_regimes,
        "durations": res.expected_durations, "transition_matrix": tm_avg}

print("Loading data...")
raw_data = load_data()
df = engineer_features(raw_data)
print(f"  {len(df)} observations ({df.index.min():%Y-%m} to {df.index.max():%Y-%m})")

print("\nFitting 2-state Markov-switching model...")
regime_df, regime_names, n_regimes, ms_info = fit_regimes(df, 2)
for r in range(n_regimes):
    sub = regime_df[regime_df["regime"] == r]
    dur = ms_info["durations"][ms_info["sorted_regimes"][r]]
    print(f"  {regime_names[r]}: N={len(sub)}, mean u={sub['unemployment'].mean():.1f}%, "
          f"mean FFR={sub['fed_funds'].mean():.1f}%, E[dur]={dur:.1f}m ({dur/12:.1f}y)")
tm = ms_info["transition_matrix"]
print(f"\n  P(stay low)={tm[0,0]:.4f}, P(stay high)={tm[1,1]:.4f}")
print(f"\nInfrastructure ready.")


In [ ]:
#!/usr/bin/env python3
"""
Component 1: Markov-Switching Regime Detection
===============================================
Uses Hamilton (1989) Markov-switching regression via statsmodels.
Now includes pre-2020 robustness check to assess COVID-era distortion.
"""









# ── Markov-Switching Model ─────────────────────────────────────────
def fit_markov_switching(df, n_regimes=2, label=""):
    prefix = f"[{label}] " if label else ""
    print(f"\n{'='*70}")
    print(f"{prefix}MARKOV-SWITCHING REGIME MODEL ({n_regimes} regimes, n={len(df)})")
    print(f"{'='*70}")

    endog = df["inflation"].copy()
    exog = sm.add_constant(df[["unemployment"]].copy())

    mod = sm.tsa.MarkovRegression(
        endog, k_regimes=n_regimes, exog=exog,
        switching_variance=True, switching_exog=True,
    )

    best_res, best_llf = None, -np.inf
    for attempt in range(8):
        try:
            res = mod.fit(search_reps=30, random_state=SEED+attempt, disp=False, maxiter=500)
            if res.llf > best_llf:
                best_llf = res.llf; best_res = res
        except Exception:
            continue

    if best_res is None:
        print("  Model failed to converge.")
        return None

    res = best_res
    print(f"\n  Log-likelihood: {res.llf:.1f}  |  AIC: {res.aic:.1f}  |  BIC: {res.bic:.1f}")

    tm = res.regime_transition
    tm_avg = tm.mean(axis=2) if tm.ndim == 3 else tm
    print(f"\n  Transition matrix (columns = from, rows = to):")
    header = "  " + " "*12 + "".join([f"  From R{j}" for j in range(n_regimes)])
    print(header)
    for i in range(n_regimes):
        row = f"  To R{i}:     "
        for j in range(n_regimes):
            row += f"  {tm_avg[i,j]:7.4f}"
        print(row)

    durations = res.expected_durations
    print(f"\n  Expected durations:")
    for r in range(n_regimes):
        print(f"    Regime {r}: {durations[r]:.1f} months ({durations[r]/12:.1f} years)")

    smoothed = res.smoothed_marginal_probabilities
    regime_df = df.copy()

    regime_means = {}
    for r in range(n_regimes):
        mask = smoothed[r] > 0.5
        regime_means[r] = df.loc[mask, "inflation"].mean() if mask.sum() > 0 else 0.0

    sorted_regimes = sorted(regime_means, key=regime_means.get)
    for new_r in range(n_regimes):
        old_r = sorted_regimes[new_r]
        regime_df[f"p_regime_{new_r}"] = smoothed[old_r].values

    regime_df["regime"] = np.argmax(
        np.column_stack([regime_df[f"p_regime_{r}"].values for r in range(n_regimes)]),
        axis=1)

    sorted_means = [regime_means[sorted_regimes[r]] for r in range(n_regimes)]
    if n_regimes == 2:
        labels = ["Low", "High"]
    elif n_regimes == 3:
        labels = ["Low", "Medium", "High"]
    else:
        labels = [f"Regime {r}" for r in range(n_regimes)]
    regime_names = {r: f"{labels[r]} inflation ({sorted_means[r]:.1f}%)" for r in range(n_regimes)}

    print(f"\n  {'Regime':<30s}  {'N':>5s}  {'pi':>7s}  {'u':>7s}  {'ff':>7s}  {'spread':>7s}  {'E[dur]':>8s}")
    print(f"  {'-'*80}")
    for r in range(n_regimes):
        sub = regime_df[regime_df["regime"] == r]
        dur = durations[sorted_regimes[r]]
        print(f"  {regime_names[r]:<30s}  {len(sub):>5d}  {sub['inflation'].mean():>7.2f}  "
              f"{sub['unemployment'].mean():>7.2f}  {sub['fed_funds'].mean():>7.2f}  "
              f"{sub['term_spread'].mean():>7.2f}  {dur:>7.1f}m")

    print(f"\n{res.summary()}")
    return res, regime_df, regime_names, n_regimes, sorted_regimes, durations


def select_n_regimes(df, max_k=4):
    print(f"\n  Model selection by BIC:")
    results = {}
    for k in range(2, max_k + 1):
        endog = df["inflation"].copy()
        exog = sm.add_constant(df[["unemployment"]].copy())
        mod = sm.tsa.MarkovRegression(endog, k_regimes=k, exog=exog,
            switching_variance=True, switching_exog=True)
        best_res, best_llf = None, -np.inf
        for attempt in range(5):
            try:
                res = mod.fit(search_reps=20, random_state=SEED+attempt, disp=False, maxiter=500)
                if res.llf > best_llf:
                    best_llf = res.llf; best_res = res
            except:
                continue
        if best_res:
            results[k] = best_res
            print(f"    k={k}: LL={best_res.llf:>8.1f}  AIC={best_res.aic:>8.1f}  BIC={best_res.bic:>8.1f}")
    if not results:
        return 2
    best_k = min(results, key=lambda k: results[k].bic)
    print(f"    -> Selected: k={best_k}")
    return best_k


# ── Pre-2020 robustness check ──────────────────────────────────────
def robustness_pre2020(df):
    """Re-estimate MS model ending sample at 2019:12 to check COVID distortion."""
    print(f"\n{'='*70}")
    print("ROBUSTNESS: PRE-2020 SAMPLE (excluding COVID-era)")
    print(f"{'='*70}")

    df_pre = df.loc[:"2019-12"].copy()
    print(f"  Pre-2020 sample: {len(df_pre)} obs ({df_pre.index.min():%Y-%m} to {df_pre.index.max():%Y-%m})")

    result_pre = fit_markov_switching(df_pre, n_regimes=2, label="Pre-2020")
    if result_pre is None:
        print("  Pre-2020 model failed.")
        return None

    _, rdf_pre, rn_pre, _, _, dur_pre = result_pre

    # Compare to full sample
    print(f"\n  --- Comparison: Pre-2020 vs Full sample ---")
    return result_pre


# ── Visualization ──────────────────────────────────────────────────
def plot_regimes(df, regime_df, regime_names, n_regimes, sorted_regimes, durations,
                 output_path, suffix=""):
    regime_colors = [C5, C1, C4, C2, "#94a3b8"][:n_regimes]
    sfx = f"_{suffix}" if suffix else ""

    # Fig 1: Smoothed probabilities
    n_panels = n_regimes + 1
    fig, axes = plt.subplots(n_panels, 1, figsize=(16, 2.5 + 2.2*n_regimes), sharex=True)
    if n_panels == 1: axes = [axes]
    fig.suptitle(f"Markov-Switching Inflation Regimes ({n_regimes} states): Smoothed Probabilities",
                 fontweight="bold", fontsize=14, y=0.995)

    ax = axes[0]; shade_recessions(ax)
    for r in range(n_regimes):
        mask = regime_df["regime"] == r
        ax.scatter(regime_df.index[mask], regime_df["inflation"][mask],
                   c=regime_colors[r], s=6, alpha=0.7, label=regime_names[r], zorder=2)
    ax.axhline(2, color="grey", ls="--", lw=1, alpha=0.5)
    ax.set_ylabel("Inflation (%)"); ax.set_title("Inflation by most likely regime", fontweight="bold")
    ax.legend(frameon=True, fontsize=7, markerscale=2, loc="upper right")

    for r in range(n_regimes):
        ax = axes[r + 1]; shade_recessions(ax)
        ax.fill_between(regime_df.index, regime_df[f"p_regime_{r}"], 0,
                        alpha=0.4, color=regime_colors[r])
        ax.plot(regime_df.index, regime_df[f"p_regime_{r}"], lw=1, color=regime_colors[r])
        ax.axhline(0.5, color="grey", ls="--", lw=0.8, alpha=0.5)
        ax.set_ylabel("P(regime)")
        dur = durations[sorted_regimes[r]]
        ax.set_title(f"P({regime_names[r]}) | E[duration]: {dur:.0f} months",
                     fontweight="bold", fontsize=10)
        ax.set_ylim(-0.05, 1.05)

    axes[-1].set_xlabel("Year")
    fig.tight_layout()
    path1 = f"{output_path}/ms_regime_probabilities{sfx}.png"
    fig.savefig(path1, dpi=200, bbox_inches="tight"); plt.close(fig)
    print(f"  Saved: {path1}")

    # Fig 2: Regime timeline + macro context
    fig, axes = plt.subplots(4, 1, figsize=(16, 12), sharex=True,
                              gridspec_kw={"height_ratios": [1, 2, 2, 2]})
    fig.suptitle("Inflation Regimes in Macroeconomic Context",
                 fontweight="bold", fontsize=14, y=0.995)

    ax = axes[0]
    for r in range(n_regimes):
        mask = regime_df["regime"] == r
        for d in regime_df.index[mask]:
            ax.axvspan(d, d + pd.DateOffset(months=1), color=regime_colors[r], alpha=0.8)
    for i, (ch, s, e) in enumerate(FED_CHAIRS):
        mid = pd.Timestamp(s) + (pd.Timestamp(e)-pd.Timestamp(s))/2
        if regime_df.index[0] <= mid <= regime_df.index[-1]:
            ax.text(mid, 0.5, ch, ha="center", va="center", fontsize=8,
                    fontstyle="italic", alpha=0.7)
    ax.set_yticks([])
    ax.set_title("Regime assignment with Fed chairs", fontweight="bold")
    legend_elements = [Patch(facecolor=regime_colors[r], alpha=0.8, label=regime_names[r])
                       for r in range(n_regimes)]
    ax.legend(handles=legend_elements, frameon=True, fontsize=7, loc="upper left",
              ncol=min(n_regimes,3))

    ax = axes[1]; shade_recessions(ax)
    ax.plot(regime_df.index, regime_df["inflation"], lw=1.5, color="#374151")
    ax.axhline(2, color=C2, ls="--", lw=1, alpha=0.5, label="2% target")
    ax.set_ylabel("Inflation (%)"); ax.set_title("YoY Inflation", fontweight="bold")
    ax.legend(frameon=False, fontsize=8)

    ax = axes[2]; shade_recessions(ax)
    ax.plot(regime_df.index, regime_df["fed_funds"], lw=1.5, color=C1, label="Fed funds")
    ax.plot(regime_df.index, regime_df["treasury_10y"], lw=1.2, color=C2, alpha=0.6, label="10Y Treasury")
    ax.set_ylabel("Rate (%)"); ax.set_title("Interest rates", fontweight="bold")
    ax.legend(frameon=False, fontsize=8)

    ax = axes[3]; shade_recessions(ax)
    ax.plot(regime_df.index, regime_df["unemployment"], lw=1.5, color=C3, label="Unemployment")
    ax.plot(regime_df.index, regime_df["nrou"], lw=1.2, color=C4, ls="--",
            alpha=0.7, label="CBO NAIRU")
    ax.set_ylabel("Rate (%)"); ax.set_xlabel("Year")
    ax.set_title("Unemployment rate vs CBO natural rate", fontweight="bold")
    ax.legend(frameon=False, fontsize=8)

    fig.tight_layout()
    path2 = f"{output_path}/ms_regime_context{sfx}.png"
    fig.savefig(path2, dpi=200, bbox_inches="tight"); plt.close(fig)
    print(f"  Saved: {path2}")

    # Fig 3: Distributions by regime
    fig, axes = plt.subplots(2, 3, figsize=(16, 8))
    fig.suptitle("Regime Characteristics", fontweight="bold", fontsize=14, y=1.01)

    for ax, (var, label) in zip(axes.flat, [
        ("inflation","Inflation (%)"), ("unemployment","Unemployment (%)"),
        ("fed_funds","Fed funds (%)"), ("real_rate","Real rate (%)"),
        ("term_spread","Term spread (pp)"), ("unemp_gap","Unemployment gap (pp, CBO NAIRU)")]):

        data_list = [regime_df.loc[regime_df["regime"]==r, var].dropna().values
                     for r in range(n_regimes)]
        bp = ax.boxplot(data_list, patch_artist=True, widths=0.6,
                        labels=[f"R{r}" for r in range(n_regimes)],
                        medianprops=dict(color="black", lw=1.5))
        for r, patch in enumerate(bp["boxes"]):
            patch.set_facecolor(regime_colors[r]); patch.set_alpha(0.6)
        ax.set_title(label, fontweight="bold", fontsize=10)
        ax.axhline(0, color="grey", lw=0.5, alpha=0.3)

    fig.tight_layout()
    path3 = f"{output_path}/ms_regime_distributions{sfx}.png"
    fig.savefig(path3, dpi=200, bbox_inches="tight"); plt.close(fig)
    print(f"  Saved: {path3}")

    return [path1, path2, path3]


# ── Data overview plot ─────────────────────────────────────────────
def plot_data_overview(df, output_path):
    fig, axes = plt.subplots(3, 2, figsize=(16, 10), sharex=True)
    fig.suptitle("US Economic Indicators, 1971-2024", fontweight="bold", fontsize=14, y=1.0)

    panels = [
        ("inflation", "CPI Inflation (%)", C1),
        ("unemployment", "Unemployment (%)", C3),
        ("fed_funds", "Fed Funds Rate (%)", C2),
        ("capacity_util", "Capacity Utilisation (%)", C4),
        ("treasury_10y", "10-Year Treasury (%)", C5),
        ("fin_conditions", "Financial Conditions (NFCI)", "#6b7280"),
    ]
    for ax, (var, title, color) in zip(axes.flat, panels):
        shade_recessions(ax)
        ax.plot(df.index, df[var], lw=1.2, color=color)
        ax.set_title(title, fontweight="bold", fontsize=10)

    # Add NAIRU to unemployment panel
    axes[0, 1].plot(df.index, df["nrou"], lw=1, color=C4, ls="--", alpha=0.6, label="CBO NAIRU")
    axes[0, 1].legend(frameon=False, fontsize=7)

    axes[-1, 0].set_xlabel("Year"); axes[-1, 1].set_xlabel("Year")
    fig.tight_layout()
    path = f"{output_path}/data_overview.png"
    fig.savefig(path, dpi=200, bbox_inches="tight"); plt.close(fig)
    print(f"  Saved: {path}")
    return path


# ── Main ───────────────────────────────────────────────────────────

# Data overview
plot_data_overview(df, OUTPUT_PATH)

# Model selection
best_k = select_n_regimes(df, max_k=4)

# Main model
result = fit_markov_switching(df, n_regimes=best_k, label="Full sample")
all_paths = []
if result is not None:
    res, regime_df, regime_names, n_regimes, sorted_regimes, durations = result
    paths = plot_regimes(df, regime_df, regime_names, n_regimes,
                        sorted_regimes, durations, OUTPUT_PATH, suffix=f"k{best_k}")
    all_paths.extend(paths)

# Always show 2-regime for comparison
if best_k != 2:
    print("\n\n--- 2-regime baseline for comparison ---")
    result2 = fit_markov_switching(df, n_regimes=2, label="2-regime baseline")
    if result2 is not None:
        res2, rdf2, rn2, n2, sr2, dur2 = result2
        paths2 = plot_regimes(df, rdf2, rn2, n2, sr2, dur2, OUTPUT_PATH, suffix="k2")
        all_paths.extend(paths2)

# Pre-2020 robustness
result_pre = robustness_pre2020(df)
if result_pre is not None:
    res_pre, rdf_pre, rn_pre, n_pre, sr_pre, dur_pre = result_pre
    paths_pre = plot_regimes(df.loc[:"2019-12"], rdf_pre, rn_pre, n_pre,
                             sr_pre, dur_pre, OUTPUT_PATH, suffix="pre2020")
    all_paths.extend(paths_pre)

print(f"\nAll outputs: {all_paths}")
print("Done.")

# ── Within-regime Phillips curves ─────────────────────────────
print("\n--- Within-regime Phillips curves (HAC SE) ---")
for r in range(n_regimes):
    rsub = regime_df[regime_df["regime"]==r][["inflation","unemp_gap"]].dropna()
    X_pc = sm.add_constant(rsub["unemp_gap"])
    res_pc = sm.OLS(rsub["inflation"], X_pc).fit(cov_type="HAC", cov_kwds={"maxlags": 12})
    print(f"  {regime_names[r]}: beta={res_pc.params['unemp_gap']:.3f}, "
          f"t={res_pc.tvalues['unemp_gap']:.2f}, R2={res_pc.rsquared:.3f}")


In [ ]:
#!/usr/bin/env python3
"""
Component 2: Regime-Conditional Taylor Rules
=============================================
Key changes from original:
- CBO NAIRU replaces hardcoded 5% unemployment gap
- Expected inflation (MICH / adaptive proxy) specification alongside realized
- HAC standard errors via block bootstrap
- Pre-2020 robustness check
"""









# ── Get regimes from Markov-switching ──────────────────────────────

# ── Taylor Rule NLS (CBO NAIRU) ───────────────────────────────────
def taylor_resid(params, y, pi, u_gap, ff_lag):
    """Residuals for structural Taylor rule using unemployment gap directly."""
    rho, r_star, a_pi, a_u = params
    target = r_star + pi + a_pi * (pi - 2.0) + a_u * u_gap
    return y - (rho * ff_lag + (1 - rho) * target)


def taylor_resid_expected(params, y, pi_e, u_gap, ff_lag):
    """Residuals using expected inflation instead of realized."""
    rho, r_star, a_pi, a_u = params
    target = r_star + pi_e + a_pi * (pi_e - 2.0) + a_u * u_gap
    return y - (rho * ff_lag + (1 - rho) * target)


def fit_taylor_nls(sub, label="", use_expected=False):
    """Fit structural Taylor rule via NLS. Uses CBO NAIRU for unemployment gap."""
    y = sub["fed_funds"].values
    u_gap = sub["unemp_gap"].values  # already unemployment - NAIRU
    ff_lag = sub["ff_lag1"].values

    if use_expected:
        pi = sub["pi_expected"].values
        valid = ~np.isnan(pi)
        if valid.sum() < 30:
            return None
        y, pi, u_gap, ff_lag = y[valid], pi[valid], u_gap[valid], ff_lag[valid]
        resid_fn = taylor_resid_expected
    else:
        pi = sub["inflation"].values
        resid_fn = taylor_resid

    res = least_squares(resid_fn, [0.85, 2.0, 0.5, 0.5],
        args=(y, pi, u_gap, ff_lag),
        bounds=([0, -5, -2, -5], [0.999, 10, 5, 5]))

    rho, rstar, a_pi, a_u = res.x
    n = len(y)
    sigma2 = np.sum(res.fun**2) / (n - 4)
    J = res.jac
    try:
        cov = sigma2 * np.linalg.inv(J.T @ J)
        se = np.sqrt(np.diag(cov))
    except:
        se = [np.nan]*4

    target = rstar + pi + a_pi * (pi - 2.0) + a_u * u_gap
    prescribed = np.maximum(target, 0.0)
    ssr = np.sum(res.fun**2)

    return {
        "label": label, "n": n, "use_expected": use_expected,
        "rho": rho, "rstar": rstar, "a_pi": a_pi, "a_u": a_u,
        "se": se, "ssr": ssr, "sigma2": sigma2,
        "prescribed": prescribed, "actual": y,
        "taylor_principle": 1 + a_pi,
    }


# ── HAC standard errors via block bootstrap ───────────────────────
def bootstrap_taylor_se(sub, n_boot=500, block_size=12, use_expected=False):
    """Block bootstrap for HAC-robust standard errors on NLS Taylor rule."""
    n = len(sub)
    boot_params = []

    for b in range(n_boot):
        # Block bootstrap indices
        n_blocks = int(np.ceil(n / block_size))
        block_starts = np.random.randint(0, n - block_size + 1, size=n_blocks)
        indices = np.concatenate([np.arange(s, s + block_size) for s in block_starts])[:n]

        boot_sub = sub.iloc[indices].copy()
        y = boot_sub["fed_funds"].values
        u_gap = boot_sub["unemp_gap"].values
        ff_lag = boot_sub["ff_lag1"].values

        if use_expected:
            pi = boot_sub["pi_expected"].values
            valid = ~np.isnan(pi)
            if valid.sum() < 30:
                continue
            y, pi, u_gap, ff_lag = y[valid], pi[valid], u_gap[valid], ff_lag[valid]
            resid_fn = taylor_resid_expected
        else:
            pi = boot_sub["inflation"].values
            resid_fn = taylor_resid

        try:
            res = least_squares(resid_fn, [0.85, 2.0, 0.5, 0.5],
                args=(y, pi, u_gap, ff_lag),
                bounds=([0, -5, -2, -5], [0.999, 10, 5, 5]),
                max_nfev=300)
            boot_params.append(res.x)
        except:
            continue

    if len(boot_params) < 50:
        return [np.nan]*4

    boot_params = np.array(boot_params)
    return np.std(boot_params, axis=0)


# ── Chow test ─────────────────────────────────────────────────────
def chow_test(df, break_date, dep="delta_ff", regressors=["inflation","unemp_gap"]):
    sub = df[[dep] + regressors].dropna()
    before = sub.loc[:break_date]
    after = sub.loc[break_date:]
    if len(before) < 10 or len(after) < 10:
        return np.nan, np.nan
    k = len(regressors) + 1
    X_pool = sm.add_constant(sub[regressors].values); y_pool = sub[dep].values
    res_pool = sm.OLS(y_pool, X_pool).fit()
    X1 = sm.add_constant(before[regressors].values); y1 = before[dep].values
    res1 = sm.OLS(y1, X1).fit()
    X2 = sm.add_constant(after[regressors].values); y2 = after[dep].values
    res2 = sm.OLS(y2, X2).fit()
    n = len(sub)
    F = ((res_pool.ssr - res1.ssr - res2.ssr) / k) / ((res1.ssr + res2.ssr) / (n - 2*k))
    p_val = 1 - stats.f.cdf(F, k, n - 2*k)
    return F, p_val


# ── Expanding-window Taylor prescription ───────────────────────────
def expanding_taylor(df, min_window=60):
    dates_out, prescribed_out, actual_out = [], [], []
    for end in range(min_window, len(df)):
        window = df.iloc[:end+1]
        current = df.iloc[end]
        try:
            y = window["fed_funds"].values
            pi = window["inflation"].values
            u_gap = window["unemp_gap"].values
            ff_lag = window["ff_lag1"].values
            res = least_squares(taylor_resid, [0.85, 2.0, 0.5, 0.5],
                args=(y, pi, u_gap, ff_lag),
                bounds=([0, -5, -2, -5], [0.999, 10, 5, 5]),
                max_nfev=500)
            rho, rstar, a_pi, a_u = res.x
            target = rstar + current["inflation"] + a_pi*(current["inflation"]-2.0) + a_u*current["unemp_gap"]
            prescribed_out.append(max(0, target))
        except:
            prescribed_out.append(np.nan)
        dates_out.append(df.index[end])
        actual_out.append(current["fed_funds"])
    return pd.DataFrame({"date": dates_out, "prescribed": prescribed_out,
                          "actual": actual_out}).set_index("date")


# ── Main analysis ──────────────────────────────────────────────────
def run_analysis(df, regime_df, regime_names, n_regimes):
    print("="*70)
    print("REGIME-CONDITIONAL TAYLOR RULES")
    print("  Using CBO NAIRU for unemployment gap")
    print("="*70)

    sub = regime_df[["fed_funds","ff_lag1","inflation","unemployment",
                     "unemp_gap","pi_expected","regime"]].dropna(
                         subset=["fed_funds","ff_lag1","inflation","unemp_gap"])

    # ── A. Full-sample Taylor rule (realized inflation) ──
    print("\n--- A. Full-sample structural Taylor rule (realized inflation, CBO NAIRU) ---")
    full = fit_taylor_nls(sub, "Full sample (realized pi)")
    print(f"\n  {'Parameter':<20s}  {'Estimate':>10s}  {'OLS SE':>8s}")
    print(f"  {'-'*42}")
    for nm, v, se in zip(["rho (smoothing)","r* (neutral)","a_pi (inflation)","a_u (unemp gap)"],
                          [full["rho"],full["rstar"],full["a_pi"],full["a_u"]], full["se"]):
        print(f"  {nm:<20s}  {v:>10.3f}  {se:>8.3f}")
    print(f"  Taylor principle: 1 + a_pi = {full['taylor_principle']:.2f}")

    # HAC standard errors
    print("\n  Computing block bootstrap HAC standard errors (500 reps)...")
    hac_se = bootstrap_taylor_se(sub, n_boot=500, block_size=12, use_expected=False)
    full["hac_se"] = hac_se
    print(f"  {'Parameter':<20s}  {'Estimate':>10s}  {'OLS SE':>8s}  {'HAC SE':>8s}")
    print(f"  {'-'*52}")
    for nm, v, se, hse in zip(["rho","r*","a_pi","a_u"],
                               [full["rho"],full["rstar"],full["a_pi"],full["a_u"]],
                               full["se"], hac_se):
        print(f"  {nm:<20s}  {v:>10.3f}  {se:>8.3f}  {hse:>8.3f}")

    # ── A2. Expected inflation specification ──
    print("\n--- A2. Full-sample Taylor rule (expected inflation, CBO NAIRU) ---")
    full_exp = fit_taylor_nls(sub, "Full sample (expected pi)", use_expected=True)
    if full_exp is not None:
        print(f"\n  {'Parameter':<20s}  {'Estimate':>10s}  {'SE':>8s}")
        print(f"  {'-'*42}")
        for nm, v, se in zip(["rho","r*","a_pi","a_u"],
                              [full_exp["rho"],full_exp["rstar"],full_exp["a_pi"],full_exp["a_u"]],
                              full_exp["se"]):
            print(f"  {nm:<20s}  {v:>10.3f}  {se:>8.3f}")
        print(f"  Taylor principle (expected pi): 1 + a_pi = {full_exp['taylor_principle']:.2f}")

        hac_se_exp = bootstrap_taylor_se(sub, n_boot=500, block_size=12, use_expected=True)
        full_exp["hac_se"] = hac_se_exp
        print(f"\n  HAC SE: {['%.3f' % s for s in hac_se_exp]}")
    else:
        print("  Not enough expected inflation data.")

    # ── B. Regime-conditional Taylor rules ──
    print(f"\n--- B. Regime-conditional Taylor rules ---")
    regime_results = {}
    regime_results_exp = {}
    for r in range(n_regimes):
        rsub = sub[sub["regime"] == r]
        if len(rsub) < 30:
            print(f"  {regime_names[r]}: too few obs ({len(rsub)}), skipping")
            continue

        # Realized inflation
        result = fit_taylor_nls(rsub, regime_names[r])
        regime_results[r] = result
        hac = bootstrap_taylor_se(rsub, n_boot=500, block_size=12)
        result["hac_se"] = hac

        # Expected inflation
        result_exp = fit_taylor_nls(rsub, f"{regime_names[r]} (exp pi)", use_expected=True)
        if result_exp is not None:
            regime_results_exp[r] = result_exp
            hac_exp = bootstrap_taylor_se(rsub, n_boot=500, block_size=12, use_expected=True)
            result_exp["hac_se"] = hac_exp

    if regime_results:
        print(f"\n  === Realized inflation specification ===")
        print(f"  {'Regime':<30s}  {'rho':>6s}  {'r*':>6s}  {'a_pi':>6s}  {'a_u':>6s}  {'1+a_pi':>7s}  {'N':>4s}")
        print(f"  {'-'*70}")
        for r, res in regime_results.items():
            tp = res["taylor_principle"]
            print(f"  {res['label']:<30s}  {res['rho']:>6.3f}  {res['rstar']:>6.3f}  "
                  f"{res['a_pi']:>6.3f}  {res['a_u']:>6.3f}  {tp:>7.2f}  {res['n']:>4d}")

        print(f"\n  HAC standard errors (block bootstrap, 12-month blocks):")
        print(f"  {'Regime':<30s}  {'se(rho)':>8s}  {'se(r*)':>8s}  {'se(a_pi)':>8s}  {'se(a_u)':>8s}")
        print(f"  {'-'*70}")
        for r, res in regime_results.items():
            hse = res["hac_se"]
            print(f"  {res['label']:<30s}  {hse[0]:>8.3f}  {hse[1]:>8.3f}  {hse[2]:>8.3f}  {hse[3]:>8.3f}")

    if regime_results_exp:
        print(f"\n  === Expected inflation specification ===")
        print(f"  {'Regime':<30s}  {'rho':>6s}  {'r*':>6s}  {'a_pi':>6s}  {'a_u':>6s}  {'1+a_pi':>7s}  {'N':>4s}")
        print(f"  {'-'*70}")
        for r, res in regime_results_exp.items():
            tp = res["taylor_principle"]
            print(f"  {res['label']:<30s}  {res['rho']:>6.3f}  {res['rstar']:>6.3f}  "
                  f"{res['a_pi']:>6.3f}  {res['a_u']:>6.3f}  {tp:>7.2f}  {res['n']:>4d}")

    # ── C. Structural break tests (using CBO NAIRU gap) ──
    print(f"\n--- C. Chow tests at regime transition dates ---")
    transitions = []
    prev_regime = regime_df["regime"].iloc[0]
    for i in range(1, len(regime_df)):
        curr = regime_df["regime"].iloc[i]
        if curr != prev_regime:
            transitions.append({
                "date": regime_df.index[i],
                "from": regime_names.get(prev_regime, f"R{prev_regime}"),
                "to": regime_names.get(curr, f"R{curr}"),
            })
        prev_regime = curr

    if transitions:
        tested_dates = set()
        print(f"\n  {'Date':<12s}  {'Transition':<40s}  {'F-stat':>8s}  {'p-value':>8s}")
        print(f"  {'-'*72}")
        for tr in transitions:
            d = tr["date"]
            if any(abs((d - td).days) < 365 for td in tested_dates):
                continue
            tested_dates.add(d)
            test_df = regime_df[["delta_ff","inflation","unemp_gap"]].dropna()
            F, p = chow_test(test_df, d.strftime("%Y-%m-%d"),
                             regressors=["inflation","unemp_gap"])
            sig = "***" if p < 0.01 else "**" if p < 0.05 else "*" if p < 0.10 else ""
            print(f"  {d:%Y-%m}      {tr['from'][:18]:>18s} -> {tr['to'][:18]:<18s}  {F:>8.2f}  {p:>7.4f} {sig}")

    # ── D. Expanding-window Taylor prescription ──
    print(f"\n--- D. Expanding-window Taylor rule prescription ---")
    expand_df = expanding_taylor(regime_df, min_window=60)
    expand_df["gap"] = expand_df["actual"] - expand_df["prescribed"]
    expand_df["regime"] = regime_df.loc[expand_df.index, "regime"]

    print(f"  Computed {len(expand_df)} real-time prescriptions")
    print(f"  Mean discretionary gap: {expand_df['gap'].mean():+.2f} pp")
    for r in range(n_regimes):
        rsub = expand_df[expand_df["regime"] == r]
        if len(rsub) > 0:
            print(f"  {regime_names[r]}: mean gap = {rsub['gap'].mean():+.2f}")

    return full, full_exp, regime_results, regime_results_exp, expand_df, transitions


# ── Visualization ──────────────────────────────────────────────────
def plot_results(regime_df, regime_names, n_regimes, full, full_exp,
                 regime_results, regime_results_exp, expand_df, output_path):
    regime_colors = [C5, C1, C4, C2, "#94a3b8"][:n_regimes]

    # ── Fig 1: Regime-conditional Taylor parameters (with HAC SE) ──
    if len(regime_results) >= 2:
        fig, axes = plt.subplots(1, 4, figsize=(16, 5))
        fig.suptitle("Taylor Rule Parameters by Inflation Regime (CBO NAIRU, HAC SE)",
                     fontweight="bold", fontsize=14, y=1.02)

        params = ["rho", "rstar", "a_pi", "a_u"]
        titles = ["Smoothing (rho)", "Neutral rate (r*)",
                  "Inflation response (a_pi)", "Unemp gap response (a_u)"]

        for ax, param, title in zip(axes, params, titles):
            labels_list = []; vals = []; colors = []; errors = []
            idx = params.index(param)

            # Full sample
            labels_list.append("Full sample"); vals.append(full[param])
            colors.append("#6b7280")
            hse = full.get("hac_se", full["se"])
            errors.append(1.96 * hse[idx] if not np.isnan(hse[idx]) else 0)

            for r in sorted(regime_results.keys()):
                res = regime_results[r]
                labels_list.append(regime_names[r][:20]); vals.append(res[param])
                colors.append(regime_colors[r])
                hse_r = res.get("hac_se", res["se"])
                errors.append(1.96 * hse_r[idx] if not np.isnan(hse_r[idx]) else 0)

            y_pos = np.arange(len(labels_list))
            ax.barh(y_pos, vals, xerr=errors, color=colors, alpha=0.75,
                    edgecolor="white", lw=1, capsize=4, height=0.6)
            ax.set_yticks(y_pos); ax.set_yticklabels(labels_list, fontsize=8)
            ax.axvline(0, color="grey", lw=0.5)
            ax.set_title(title, fontweight="bold", fontsize=10)
            ax.invert_yaxis()
            if param == "a_pi":
                ax.axvline(0, color=C2, ls="--", lw=1, alpha=0.5)

        fig.tight_layout()
        path1 = f"{output_path}/taylor_by_regime.png"
        fig.savefig(path1, dpi=200, bbox_inches="tight"); plt.close(fig)
        print(f"  Saved: {path1}")
    else:
        path1 = None

    # ── Fig 2: Taylor prescription vs actual ──
    fig, axes = plt.subplots(3, 1, figsize=(16, 12), sharex=True,
                              gridspec_kw={"height_ratios": [1, 3, 2]})
    fig.suptitle("Taylor Rule Prescription vs Actual Fed Funds Rate (CBO NAIRU)",
                 fontweight="bold", fontsize=14, y=0.995)

    ax = axes[0]
    for r in range(n_regimes):
        mask = regime_df.loc[expand_df.index, "regime"] == r
        for d in expand_df.index[mask]:
            ax.axvspan(d, d + pd.DateOffset(months=1), color=regime_colors[r], alpha=0.8)
    ax.set_yticks([])
    ax.set_title("Inflation regime", fontweight="bold", fontsize=10)
    legend_elements = [Patch(facecolor=regime_colors[r], alpha=0.8, label=regime_names[r])
                       for r in range(n_regimes)]
    ax.legend(handles=legend_elements, frameon=True, fontsize=7, loc="upper left",
              ncol=min(n_regimes,3))

    ax = axes[1]; shade_recessions(ax)
    ax.plot(expand_df.index, expand_df["actual"], lw=2, color=C1, label="Actual fed funds", zorder=3)
    ax.plot(expand_df.index, expand_df["prescribed"], lw=1.5, color=C2, alpha=0.8,
            label="Taylor rule prescription", zorder=2)
    ax.fill_between(expand_df.index, expand_df["actual"], expand_df["prescribed"],
                    where=expand_df["actual"] > expand_df["prescribed"],
                    alpha=0.15, color=C2, label="Too tight")
    ax.fill_between(expand_df.index, expand_df["actual"], expand_df["prescribed"],
                    where=expand_df["actual"] < expand_df["prescribed"],
                    alpha=0.15, color=C1, label="Too loose")
    ax.set_ylabel("Rate (%)")
    ax.set_title("Fed funds: actual vs Taylor rule prescription", fontweight="bold")
    ax.legend(frameon=True, fontsize=8, loc="upper right")
    ax.set_ylim(bottom=-1)

    ax = axes[2]; shade_recessions(ax)
    gap = expand_df["gap"]
    ax.fill_between(expand_df.index, gap, 0, where=gap > 0, alpha=0.4, color=C2, label="Tighter than rule")
    ax.fill_between(expand_df.index, gap, 0, where=gap < 0, alpha=0.4, color=C1, label="Looser than rule")
    ax.axhline(0, color="grey", lw=1)
    ax.set_ylabel("Gap (pp)"); ax.set_xlabel("Year")
    ax.set_title("Discretionary gap (actual minus prescribed)", fontweight="bold")
    ax.legend(frameon=True, fontsize=8)

    fig.tight_layout()
    path2 = f"{output_path}/taylor_prescription_vs_actual.png"
    fig.savefig(path2, dpi=200, bbox_inches="tight"); plt.close(fig)
    print(f"  Saved: {path2}")

    # ── Fig 3: Taylor principle by regime (realized vs expected) ──
    if len(regime_results) >= 2:
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        fig.suptitle("Taylor Principle: Realized vs Expected Inflation",
                     fontweight="bold", fontsize=14, y=1.02)

        for ax, (results_dict, title_suffix) in zip(axes, [
            (regime_results, "Realized inflation"),
            (regime_results_exp, "Expected inflation (MICH/adaptive)")]):

            if not results_dict:
                ax.text(0.5, 0.5, "Not enough data", ha="center", va="center",
                        transform=ax.transAxes)
                ax.set_title(title_suffix, fontweight="bold")
                continue

            labels_list = ["Full sample"] + [regime_names[r][:25] for r in sorted(results_dict.keys())]
            ref = full if "Realized" in title_suffix else full_exp
            if ref is None:
                continue
            tp_vals = [ref["taylor_principle"]] + \
                      [results_dict[r]["taylor_principle"] for r in sorted(results_dict.keys())]
            colors_bar = ["#6b7280"] + [regime_colors[r] for r in sorted(results_dict.keys())]

            # HAC-based error bars
            errs = []
            hse = ref.get("hac_se", ref["se"])
            errs.append(1.96 * hse[2] if not np.isnan(hse[2]) else 0)
            for r in sorted(results_dict.keys()):
                hse_r = results_dict[r].get("hac_se", results_dict[r]["se"])
                errs.append(1.96 * hse_r[2] if not np.isnan(hse_r[2]) else 0)

            bars = ax.bar(labels_list, tp_vals, yerr=errs, color=colors_bar,
                          alpha=0.75, edgecolor="white", lw=1, width=0.6, capsize=5)
            ax.axhline(1.0, color=C2, ls="--", lw=2, alpha=0.7, label="Taylor principle = 1")
            ax.set_ylabel("1 + a_pi")
            ax.set_title(title_suffix, fontweight="bold")
            ax.legend(frameon=False, fontsize=9)
            for bar, val in zip(bars, tp_vals):
                ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(errs)*0.1 + 0.02,
                        f"{val:.2f}", ha="center", va="bottom", fontsize=10, fontweight="bold")

        fig.tight_layout()
        path3 = f"{output_path}/taylor_principle_by_regime.png"
        fig.savefig(path3, dpi=200, bbox_inches="tight"); plt.close(fig)
        print(f"  Saved: {path3}")
    else:
        path3 = None

    return [p for p in [path1, path2, path3] if p]


# ── Main ───────────────────────────────────────────────────────────


full, full_exp, regime_results, regime_results_exp, expand_df, transitions = \
    run_analysis(df, regime_df, regime_names, n_regimes)

paths = plot_results(regime_df, regime_names, n_regimes, full, full_exp,
                     regime_results, regime_results_exp, expand_df, OUTPUT_PATH)

print(f"\nOutputs: {paths}")
print("Done.")

# ── Granger causality: inflation -> delta_ff ──────────────────
print("\n--- Granger causality: inflation -> delta_ff ---")
gdf = regime_df[["delta_ff","inflation"]].dropna()
for ml in [3, 6, 12]:
    y_g = gdf["delta_ff"].values
    Xr = sm.add_constant(np.column_stack([gdf["delta_ff"].shift(l).values for l in range(1,ml+1)]))
    Xu = sm.add_constant(np.column_stack(
        [gdf["delta_ff"].shift(l).values for l in range(1,ml+1)] +
        [gdf["inflation"].shift(l).values for l in range(1,ml+1)]))
    v = ~np.isnan(Xu).any(axis=1) & ~np.isnan(y_g)
    rr_ = sm.OLS(y_g[v], Xr[v]).fit()
    ru_ = sm.OLS(y_g[v], Xu[v]).fit()
    Fg = ((rr_.ssr - ru_.ssr)/ml) / (ru_.ssr/(v.sum() - Xu.shape[1]))
    pg = 1 - stats.f.cdf(Fg, ml, v.sum() - Xu.shape[1])
    sg = "***" if pg<0.01 else "**" if pg<0.05 else "*" if pg<0.10 else ""
    print(f"  {ml} lags: F={Fg:.2f}, p={pg:.4f} {sg}")


In [ ]:
#!/usr/bin/env python3
"""
Component 3: Structural Break Detection & Yield Curve Analysis
==============================================================
Updated: uses CBO NAIRU-based unemployment gap in break detection.
"""










# ================================================================
# PART A: STRUCTURAL BREAK DETECTION (now uses CBO NAIRU gap)
# ================================================================
def sup_wald_break_test(df, dep="delta_ff", regressors=["inflation","unemp_gap"],
                        min_frac=0.15):
    sub = df[[dep] + regressors].dropna()
    n = len(sub); trim = int(n * min_frac); k = len(regressors) + 1
    X_pool = sm.add_constant(sub[regressors].values)
    y_pool = sub[dep].values
    res_pool = sm.OLS(y_pool, X_pool).fit()
    ssr_pool = res_pool.ssr

    dates_test, f_stats = [], []
    for t in range(trim, n - trim):
        X1 = sm.add_constant(sub[regressors].values[:t])
        y1 = sub[dep].values[:t]
        X2 = sm.add_constant(sub[regressors].values[t:])
        y2 = sub[dep].values[t:]
        try:
            res1 = sm.OLS(y1, X1).fit(); res2 = sm.OLS(y2, X2).fit()
            F = ((ssr_pool - res1.ssr - res2.ssr) / k) / ((res1.ssr + res2.ssr) / (n - 2*k))
            f_stats.append(F)
        except:
            f_stats.append(0)
        dates_test.append(sub.index[t])

    f_stats = np.array(f_stats)
    dates_test = pd.DatetimeIndex(dates_test)
    best_idx = np.argmax(f_stats)

    # Andrews (1993) critical values for sup-Wald, k=3, 15% trimming
    cv_10 = 7.12; cv_05 = 8.68; cv_01 = 12.16
    best_F = f_stats[best_idx]
    sig = "***" if best_F > cv_01 else "**" if best_F > cv_05 else "*" if best_F > cv_10 else ""

    return {
        "best_date": dates_test[best_idx], "best_F": best_F, "sig": sig,
        "cv": {"1%": cv_01, "5%": cv_05, "10%": cv_10},
        "dates": dates_test, "f_stats": f_stats,
    }


def sequential_breaks(df, dep="delta_ff", regressors=["inflation","unemp_gap"],
                      max_breaks=4, min_frac=0.15):
    print(f"\n{'='*70}")
    print("STRUCTURAL BREAK DETECTION (Sequential sup-Wald)")
    print(f"  Using CBO NAIRU-based unemployment gap")
    print(f"{'='*70}")
    print(f"  DV: {dep}  |  Regressors: {regressors}")

    sub = df[[dep] + regressors].dropna()
    breaks = []
    segments = [(sub.index[0], sub.index[-1])]

    for b in range(max_breaks):
        best_overall = None
        for seg_start, seg_end in segments:
            seg = sub.loc[seg_start:seg_end]
            if len(seg) < int(len(sub) * min_frac * 2):
                continue
            result = sup_wald_break_test(seg, dep, regressors, min_frac)
            if result["best_F"] > result["cv"]["5%"]:
                if best_overall is None or result["best_F"] > best_overall["best_F"]:
                    best_overall = result
                    best_overall["segment"] = (seg_start, seg_end)

        if best_overall is None:
            print(f"\n  Break {b+1}: no further significant breaks found.")
            break

        breaks.append(best_overall)
        bd = best_overall["best_date"]
        print(f"\n  Break {b+1}: {bd:%Y-%m}  (F = {best_overall['best_F']:.2f}{best_overall['sig']})")

        seg_s, seg_e = best_overall["segment"]
        new_segments = []
        for s, e in segments:
            if s == seg_s and e == seg_e:
                new_segments.append((s, bd)); new_segments.append((bd, e))
            else:
                new_segments.append((s, e))
        segments = new_segments

    print(f"\n  Detected breaks vs chair transitions:")
    print(f"  {'Break date':<14s}  {'Nearest chair transition':<30s}  {'Distance':>10s}")
    print(f"  {'-'*58}")
    for brk in breaks:
        bd = brk["best_date"]
        nearest = min(CHAIR_DATES, key=lambda x: min(abs((bd-x[1]).days), abs((bd-x[2]).days)))
        d1 = abs((bd - nearest[1]).days); d2 = abs((bd - nearest[2]).days)
        if d1 < d2:
            chair_event = f"{nearest[0]} start ({nearest[1]:%Y-%m})"; dist_months = d1/30
        else:
            chair_event = f"{nearest[0]} end ({nearest[2]:%Y-%m})"; dist_months = d2/30
        print(f"  {bd:%Y-%m}        {chair_event:<30s}  {dist_months:>8.1f}m")

    return breaks, segments


# ================================================================
# PART B: YIELD CURVE AND REGIME TRANSITIONS
# ================================================================
def yield_curve_regime_analysis(regime_df, regime_names, n_regimes):
    print(f"\n{'='*70}")
    print("YIELD CURVE AND REGIME TRANSITIONS")
    print(f"{'='*70}")

    transitions = []
    prev = regime_df["regime"].iloc[0]
    for i in range(1, len(regime_df)):
        curr = regime_df["regime"].iloc[i]
        if curr != prev:
            transitions.append({"date": regime_df.index[i], "from_regime": prev,
                                "to_regime": curr,
                                "direction": "up" if curr > prev else "down"})
        prev = curr

    filtered = []
    for tr in transitions:
        if not filtered or (tr["date"] - filtered[-1]["date"]).days > 365:
            filtered.append(tr)
    transitions = filtered

    print(f"\n  Major regime transitions: {len(transitions)}")
    print(f"\n  {'Date':<12s}  {'Direction':<8s}  {'Spread at t':>12s}  {'Spread t-6':>12s}  {'Spread t-12':>12s}")
    print(f"  {'-'*60}")

    for tr in transitions:
        d = tr["date"]
        spread_at = regime_df.loc[:d, "term_spread"].iloc[-1]
        d_6 = d - pd.DateOffset(months=6)
        mask_6 = (regime_df.index >= d_6) & (regime_df.index < d)
        spread_6 = regime_df.loc[mask_6, "term_spread"].mean() if mask_6.sum() > 0 else np.nan
        d_12 = d - pd.DateOffset(months=12)
        mask_12 = (regime_df.index >= d_12) & (regime_df.index < d)
        spread_12 = regime_df.loc[mask_12, "term_spread"].mean() if mask_12.sum() > 0 else np.nan
        tr["spread_at"] = spread_at; tr["spread_6m"] = spread_6; tr["spread_12m"] = spread_12
        print(f"  {d:%Y-%m}      {tr['direction']:<8s}  {spread_at:>12.2f}  {spread_6:>12.2f}  {spread_12:>12.2f}")

    # Logit test
    rdf = regime_df.copy()
    rdf["regime_change"] = (rdf["regime"].diff() != 0).astype(int)
    rdf["regime_change_12m"] = rdf["regime_change"].rolling(12).max().shift(-12)
    test_df = rdf[["term_spread","regime_change_12m"]].dropna()
    if len(test_df) > 50:
        X = sm.add_constant(test_df["term_spread"].values)
        y = test_df["regime_change_12m"].values
        try:
            logit = sm.Logit(y, X).fit(disp=False)
            print(f"\n  Logit: P(regime change in 12m) ~ term_spread")
            print(f"  Spread coefficient: {logit.params[1]:.4f} (p={logit.pvalues[1]:.4f})")
            if logit.pvalues[1] < 0.05:
                print(f"  -> Term spread significantly predicts regime transitions")
            else:
                print(f"  -> Term spread does NOT significantly predict regime transitions")
        except:
            pass

    return transitions


# ================================================================
# PART C: RECESSION MODEL ROBUSTNESS
# ================================================================
def leave_one_recession_out(df):
    print(f"\n{'='*70}")
    print("RECESSION MODEL: LEAVE-ONE-RECESSION-OUT CV")
    print(f"{'='*70}")

    rec_features = ["term_spread","inflation","unemp_gap","fed_funds",
                    "capacity_gap","fin_conditions","real_rate"]

    df = df.copy()
    for h in [6,12,18]:
        col = f"recession_{h}m_ahead"
        if col not in df.columns:
            df[col] = df["recession"].rolling(h).max().shift(-h)

    recession_episodes = [
        ("1973-74 Oil", "1973-11", "1975-03"),
        ("1980 Volcker I", "1980-01", "1980-07"),
        ("1981-82 Volcker II", "1981-07", "1982-11"),
        ("1990-91", "1990-07", "1991-03"),
        ("2001 Dot-com", "2001-03", "2001-11"),
        ("2007-09 GFC", "2007-12", "2009-06"),
        ("2020 COVID", "2020-02", "2020-04"),
    ]

    for h in [6, 12]:
        target = f"recession_{h}m_ahead"
        rec_df = df[rec_features + [target]].dropna()
        y_full = rec_df[target].astype(int).values
        X_full = rec_df[rec_features].values

        print(f"\n  --- {h}-month horizon ---")
        print(f"  {'Left out':<22s}  {'Train N':>8s}  {'Test N':>8s}  {'AUC (full)':>10s}  {'AUC (spread)':>12s}")
        print(f"  {'-'*66}")

        tr_mask = rec_df.index <= "2015-12"; te_mask = ~tr_mask
        if te_mask.sum() > 0 and len(np.unique(y_full[te_mask])) > 1:
            sc = StandardScaler(); X_tr = sc.fit_transform(X_full[tr_mask]); X_te = sc.transform(X_full[te_mask])
            clf = LogisticRegression(C=1.0, max_iter=1000, random_state=SEED)
            clf.fit(X_tr, y_full[tr_mask])
            ref_auc = roc_auc_score(y_full[te_mask], clf.predict_proba(X_te)[:,1])
            print(f"  {'Time-split ref':<22s}  {tr_mask.sum():>8d}  {te_mask.sum():>8d}  {ref_auc:>10.3f}")
        print()

        results_loro = []
        for name, rs, re in recession_episodes:
            pre_start = pd.Timestamp(rs) - pd.DateOffset(months=18)
            test_mask = (rec_df.index >= pre_start) & (rec_df.index <= re)
            train_mask = ~test_mask
            y_tr = y_full[train_mask]; X_tr_raw = X_full[train_mask]
            y_te = y_full[test_mask]; X_te_raw = X_full[test_mask]
            if len(y_te) < 5 or len(np.unique(y_te)) < 2:
                print(f"  {name:<22s}  {train_mask.sum():>8d}  {test_mask.sum():>8d}  {'skip':>10s}")
                continue
            sc = StandardScaler()
            X_tr = sc.fit_transform(X_tr_raw); X_te = sc.transform(X_te_raw)
            clf_full = LogisticRegression(C=1.0, max_iter=1000, random_state=SEED)
            clf_full.fit(X_tr, y_tr)
            auc_full = roc_auc_score(y_te, clf_full.predict_proba(X_te)[:,1])
            si = rec_features.index("term_spread")
            clf_sp = LogisticRegression(C=1.0, max_iter=1000, random_state=SEED)
            clf_sp.fit(X_tr[:,si:si+1], y_tr)
            auc_sp = roc_auc_score(y_te, clf_sp.predict_proba(X_te[:,si:si+1])[:,1])
            results_loro.append({"name": name, "auc_full": auc_full, "auc_spread": auc_sp})
            print(f"  {name:<22s}  {train_mask.sum():>8d}  {test_mask.sum():>8d}  {auc_full:>10.3f}  {auc_sp:>12.3f}")

        if results_loro:
            avg_full = np.mean([r["auc_full"] for r in results_loro])
            avg_sp = np.mean([r["auc_spread"] for r in results_loro])
            print(f"\n  {'Average LORO':<22s}  {'':>8s}  {'':>8s}  {avg_full:>10.3f}  {avg_sp:>12.3f}")

    return results_loro


# ================================================================
# VISUALIZATION
# ================================================================
def plot_results(df, regime_df, regime_names, n_regimes, breaks, transitions, output_path):
    regime_colors = [C5, C1, C4, C2][:n_regimes]
    chair_colors_map = {"Burns":"#dbeafe","Miller":"#fee2e2","Volcker":"#ede9fe",
                        "Greenspan":"#fef3c7","Bernanke":"#d1fae5","Yellen":"#fce7f3","Powell":"#e0e7ff"}

    # ── Fig 1: Sup-Wald F-statistic ──
    if breaks:
        first_break = breaks[0]
        fig, axes = plt.subplots(3, 1, figsize=(16, 10), sharex=True,
                                  gridspec_kw={"height_ratios": [1, 2, 2]})
        fig.suptitle("Structural Break Detection in Fed Reaction Function (CBO NAIRU gap)",
                     fontweight="bold", fontsize=14, y=0.995)

        ax = axes[0]
        for ch, cs, ce in CHAIR_DATES:
            color = chair_colors_map.get(ch, "#e5e7eb")
            ax.axvspan(cs, ce, color=color, alpha=0.5)
            mid = cs + (ce - cs)/2
            if df.index[0] <= mid <= df.index[-1]:
                ax.text(mid, 0.5, ch, ha="center", va="center", fontsize=8, fontstyle="italic")
        ax.set_yticks([]); ax.set_title("Fed chair tenures", fontweight="bold", fontsize=10)

        ax = axes[1]
        ax.plot(first_break["dates"], first_break["f_stats"], lw=2, color=C1)
        ax.axhline(first_break["cv"]["5%"], color=C2, ls="--", lw=1.5,
                    label=f'5% critical value ({first_break["cv"]["5%"]:.1f})')
        ax.axhline(first_break["cv"]["1%"], color=C2, ls=":", lw=1,
                    label=f'1% critical value ({first_break["cv"]["1%"]:.1f})')
        for brk in breaks:
            ax.axvline(brk["best_date"], color=C3, ls="-", lw=2, alpha=0.7)
            ax.text(brk["best_date"], ax.get_ylim()[1]*0.9,
                    f' {brk["best_date"]:%Y-%m}', fontsize=9, color=C3, fontweight="bold")
        for ch, cs, ce in CHAIR_DATES:
            if first_break["dates"][0] <= cs <= first_break["dates"][-1]:
                ax.axvline(cs, color="#9ca3af", ls="--", lw=1, alpha=0.5)
        ax.set_ylabel("F-statistic")
        ax.set_title("Sup-Wald F-statistic (testing every possible break date)", fontweight="bold")
        ax.legend(frameon=True, fontsize=8)

        ax = axes[2]; shade_recessions(ax)
        test_df = df[["delta_ff"]].dropna()
        ax.plot(test_df.index, test_df["delta_ff"], lw=1, color="#374151", alpha=0.7)
        for brk in breaks:
            ax.axvline(brk["best_date"], color=C3, ls="-", lw=2, alpha=0.7)
        ax.axhline(0, color="grey", lw=0.8)
        ax.set_ylabel("Monthly change (pp)")
        ax.set_title("Fed funds rate changes (dependent variable)", fontweight="bold")
        ax.set_xlabel("Year")

        fig.tight_layout()
        path1 = f"{output_path}/structural_breaks.png"
        fig.savefig(path1, dpi=200, bbox_inches="tight"); plt.close(fig)
        print(f"  Saved: {path1}")
    else:
        path1 = None

    # ── Fig 2: Term spread around regime transitions ──
    if transitions:
        fig, axes = plt.subplots(2, 1, figsize=(16, 8), sharex=True)
        fig.suptitle("Yield Curve Behaviour Around Inflation Regime Transitions",
                     fontweight="bold", fontsize=14, y=0.995)

        ax = axes[0]; shade_recessions(ax)
        ax.plot(regime_df.index, regime_df["term_spread"], lw=1.5, color=C1)
        ax.axhline(0, color=C2, ls="--", lw=1.5, alpha=0.7, label="Inversion threshold")
        for tr in transitions:
            color = C2 if tr["direction"] == "up" else C5
            ax.axvline(tr["date"], color=color, ls="-", lw=2, alpha=0.7)
        ax.set_ylabel("Term spread (pp)")
        ax.set_title("10Y-FFR spread with regime transition dates", fontweight="bold")
        legend_elements = [
            Line2D([0],[0], color=C2, ls="-", lw=2, label="Transition to high inflation"),
            Line2D([0],[0], color=C5, ls="-", lw=2, label="Transition to low inflation"),
            Line2D([0],[0], color=C2, ls="--", lw=1.5, label="Inversion threshold"),
        ]
        ax.legend(handles=legend_elements, frameon=True, fontsize=8)

        ax = axes[1]
        for r in range(n_regimes):
            mask = regime_df["regime"] == r
            for d in regime_df.index[mask]:
                ax.axvspan(d, d + pd.DateOffset(months=1), color=regime_colors[r], alpha=0.8)
        ax.set_yticks([]); ax.set_xlabel("Year")
        ax.set_title("Inflation regime", fontweight="bold")
        legend_elements = [Patch(facecolor=regime_colors[r], alpha=0.8, label=regime_names[r])
                           for r in range(n_regimes)]
        ax.legend(handles=legend_elements, frameon=True, fontsize=7, loc="upper left")

        fig.tight_layout()
        path2 = f"{output_path}/yield_curve_regimes.png"
        fig.savefig(path2, dpi=200, bbox_inches="tight"); plt.close(fig)
        print(f"  Saved: {path2}")
    else:
        path2 = None

    return [p for p in [path1, path2] if p]


# ── Main ───────────────────────────────────────────────────────────


breaks, segments = sequential_breaks(
    regime_df, dep="delta_ff", regressors=["inflation","unemp_gap"],
    max_breaks=3, min_frac=0.15)

transitions = yield_curve_regime_analysis(regime_df, regime_names, n_regimes)
loro_results = leave_one_recession_out(df)

paths = plot_results(df, regime_df, regime_names, n_regimes,
                     breaks, transitions, OUTPUT_PATH)

print(f"\nOutputs: {paths}")
print("Done.")

In [ ]:
#!/usr/bin/env python3
"""
Component 4: Regime-Aware Inflation Forecasting
================================================
Updated: uses CBO NAIRU-based unemployment gap throughout.
Removed synthetic data fallback.
"""










# ================================================================
# FORECASTING
# ================================================================

BASE_FEATURES = ["L1_inflation","L3_inflation","L6_inflation","L12_inflation"]
EXTENDED_FEATURES = BASE_FEATURES + [
    "L1_fed_funds","L3_fed_funds","L6_fed_funds",
    "L6_unemp_gap","L6_term_spread","L6_fin_conditions"
]

CV_FOLDS = [
    {"train_end":"1990-12","test_start":"1991-01","test_end":"1995-12","name":"Early 1990s"},
    {"train_end":"1995-12","test_start":"1996-01","test_end":"2000-12","name":"Late 1990s"},
    {"train_end":"2000-12","test_start":"2001-01","test_end":"2007-12","name":"2000s"},
    {"train_end":"2007-12","test_start":"2008-01","test_end":"2015-12","name":"GFC & after"},
    {"train_end":"2015-12","test_start":"2016-01","test_end":"2023-01","name":"Recent"},
]

HORIZONS = [1, 3, 6, 12]


def run_forecasting(regime_df, regime_names, n_regimes):
    print(f"\n{'='*70}")
    print("REGIME-AWARE INFLATION FORECASTING (CBO NAIRU gap)")
    print(f"{'='*70}")

    base_f = [f for f in BASE_FEATURES if f in regime_df.columns]
    ext_f = [f for f in EXTENDED_FEATURES if f in regime_df.columns]

    all_results = {}

    for h in HORIZONS:
        target = f"inflation_{h}m_ahead"
        if target not in regime_df.columns:
            regime_df[target] = regime_df["inflation"].shift(-h)

        fc_df = regime_df[ext_f + [target, "regime"] +
                          [f"p_regime_{r}" for r in range(n_regimes)]].dropna()

        print(f"\n  === {h}-month horizon ({len(fc_df)} obs) ===")

        models = {
            "Random walk": {"features": None, "regime_aware": False, "fn": None},
        "AR (lags only)": {"features": base_f, "regime_aware": False,
                "fn": lambda: Ridge(alpha=1.0)},
            "Pooled Ridge": {"features": ext_f, "regime_aware": False,
                "fn": lambda: Ridge(alpha=10.0)},
            "Ridge + regime": {"features": ext_f + ["regime"], "regime_aware": False,
                "fn": lambda: Ridge(alpha=10.0)},
            "Ridge + regime prob": {"features": ext_f + [f"p_regime_{r}" for r in range(n_regimes)],
                "regime_aware": False, "fn": lambda: Ridge(alpha=10.0)},
            "Regime-conditional": {"features": ext_f, "regime_aware": True,
                "fn": lambda: Ridge(alpha=10.0)},
            "RF pooled": {"features": ext_f, "regime_aware": False,
                "fn": lambda: RandomForestRegressor(n_estimators=200, max_depth=6,
                    min_samples_leaf=10, random_state=SEED)},
        }

        horizon_results = {}

        for mname, spec in models.items():
            fold_results = []

            for fold in CV_FOLDS:
                tr = fc_df.loc[:fold["train_end"]]
                te = fc_df.loc[fold["test_start"]:fold["test_end"]]
                if len(te) < 6 or len(tr) < 30:
                    continue

                # Random walk benchmark
                if spec["fn"] is None:
                    preds = te["inflation"].values
                    fold_results.append({"mse": mean_squared_error(te[target].values, preds),
                        "r2": r2_score(te[target].values, preds),
                        "mae": mean_absolute_error(te[target].values, preds),
                        "fold": fold["name"]})
                    continue

                feats = [f for f in spec["features"] if f in fc_df.columns]

                if spec["regime_aware"]:
                    preds = np.full(len(te), np.nan)
                    for r in range(n_regimes):
                        tr_r = tr[tr["regime"] == r]
                        te_r_mask = te["regime"] == r
                        if te_r_mask.sum() == 0:
                            continue
                        if len(tr_r) < 15:
                            sc = StandardScaler()
                            Xtr = sc.fit_transform(tr[feats])
                            Xte = sc.transform(te.loc[te_r_mask, feats])
                            m = spec["fn"](); m.fit(Xtr, tr[target].values)
                            preds[te_r_mask.values] = m.predict(Xte)
                            continue
                        sc = StandardScaler()
                        Xtr = sc.fit_transform(tr_r[feats])
                        Xte = sc.transform(te.loc[te_r_mask, feats])
                        m = spec["fn"](); m.fit(Xtr, tr_r[target].values)
                        preds[te_r_mask.values] = m.predict(Xte)

                    valid = ~np.isnan(preds)
                    if valid.sum() < 6: continue
                    mse = mean_squared_error(te[target].values[valid], preds[valid])
                    r2 = r2_score(te[target].values[valid], preds[valid])
                    mae = mean_absolute_error(te[target].values[valid], preds[valid])
                else:
                    sc = StandardScaler()
                    Xtr = sc.fit_transform(tr[feats])
                    Xte = sc.transform(te[feats])
                    m = spec["fn"](); m.fit(Xtr, tr[target].values)
                    preds = m.predict(Xte)
                    mse = mean_squared_error(te[target].values, preds)
                    r2 = r2_score(te[target].values, preds)
                    mae = mean_absolute_error(te[target].values, preds)

                fold_results.append({"mse": mse, "r2": r2, "mae": mae, "fold": fold["name"]})

            if fold_results:
                horizon_results[mname] = {
                    "avg_mse": np.mean([f["mse"] for f in fold_results]),
                    "avg_r2": np.mean([f["r2"] for f in fold_results]),
                    "avg_mae": np.mean([f["mae"] for f in fold_results]),
                    "folds": fold_results
                }

        print(f"\n  {'Model':<25s}  {'MSE':>8s}  {'MAE':>8s}  {'R2':>8s}")
        print(f"  {'-'*53}")
        for mname in models:
            if mname in horizon_results:
                r = horizon_results[mname]
                print(f"  {mname:<25s}  {r['avg_mse']:>8.3f}  {r['avg_mae']:>8.3f}  {r['avg_r2']:>+8.3f}")

        all_results[h] = horizon_results

    return all_results


# ================================================================
# SUBSAMPLE STABILITY
# ================================================================
def subsample_stability(regime_df):
    print(f"\n{'='*70}")
    print("SUBSAMPLE STABILITY: WHY 12-MONTH FORECASTING FAILS")
    print(f"{'='*70}")

    print(f"\n  Inflation autocorrelation by regime:")
    print(f"  {'Regime':<25s}", end="")
    for lag in [1, 3, 6, 12]:
        print(f"  {'AC('+str(lag)+')':>7s}", end="")
    print()
    print(f"  {'-'*60}")

    for r in sorted(regime_df["regime"].unique()):
        sub = regime_df[regime_df["regime"] == r]["inflation"]
        print(f"  {'R'+str(r)+' (n='+str(len(sub))+')' :<25s}", end="")
        for lag in [1, 3, 6, 12]:
            ac = sub.autocorr(lag=lag)
            print(f"  {ac:>7.3f}", end="")
        print()

    full = regime_df["inflation"]
    print(f"  {'Full sample':<25s}", end="")
    for lag in [1, 3, 6, 12]:
        print(f"  {full.autocorr(lag=lag):>7.3f}", end="")
    print()

    ext_f = [f for f in EXTENDED_FEATURES if f in regime_df.columns]
    target = "inflation_12m_ahead"
    if target not in regime_df.columns:
        regime_df[target] = regime_df["inflation"].shift(-12)
    fc_df = regime_df[ext_f + [target]].dropna()

    periods = [
        ("Pre-Volcker", "1973-01", "1979-12"),
        ("Volcker disinflation", "1980-01", "1986-12"),
        ("Great Moderation I", "1987-01", "1999-12"),
        ("Great Moderation II", "2000-01", "2007-12"),
        ("Post-GFC", "2008-01", "2019-12"),
        ("COVID+", "2020-01", "2023-01"),
    ]

    print(f"\n  {'Period':<25s}  {'N':>5s}  {'Var(pi)':>8s}  {'AR R2':>8s}  {'Ridge R2':>9s}")
    print(f"  {'-'*60}")

    period_results = []
    for pname, ps, pe in periods:
        tr = fc_df.loc[:ps]; te = fc_df.loc[ps:pe]
        if len(tr) < 30 or len(te) < 6: continue
        var_pi = te[target].var()

        base_f = [f for f in BASE_FEATURES if f in fc_df.columns]
        sc = StandardScaler()
        Xtr = sc.fit_transform(tr[base_f]); Xte = sc.transform(te[base_f])
        m_ar = Ridge(alpha=1.0); m_ar.fit(Xtr, tr[target].values)
        r2_ar = r2_score(te[target].values, m_ar.predict(Xte))

        sc2 = StandardScaler()
        Xtr2 = sc2.fit_transform(tr[ext_f]); Xte2 = sc2.transform(te[ext_f])
        m_ridge = Ridge(alpha=10.0); m_ridge.fit(Xtr2, tr[target].values)
        r2_ridge = r2_score(te[target].values, m_ridge.predict(Xte2))

        period_results.append({"period": pname, "n": len(te), "var_pi": var_pi,
                               "r2_ar": r2_ar, "r2_ridge": r2_ridge})
        print(f"  {pname:<25s}  {len(te):>5d}  {var_pi:>8.2f}  {r2_ar:>+8.3f}  {r2_ridge:>+9.3f}")

    return period_results


# ================================================================
# VISUALIZATION
# ================================================================
def plot_results(all_results, period_results, output_path):
    model_colors = {"AR (lags only)": "#6b7280", "Pooled Ridge": C1,
                    "Ridge + regime": C3, "Ridge + regime prob": C4,
                    "Regime-conditional": C2, "RF pooled": C5}

    # ── Fig 1: MSE and R2 by horizon ──
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle("Inflation Forecasting: Does Regime Awareness Help?",
                 fontweight="bold", fontsize=14, y=1.02)

    model_names = list(all_results[HORIZONS[0]].keys())
    x = np.arange(len(HORIZONS)); w = 0.8 / len(model_names)

    for i, mname in enumerate(model_names):
        mses = [all_results[h].get(mname, {}).get("avg_mse", np.nan) for h in HORIZONS]
        color = model_colors.get(mname, f"C{i}")
        axes[0].bar(x + i*w, mses, w, label=mname, color=color, alpha=0.75,
                    edgecolor="white", lw=1)
    axes[0].set_xticks(x + w*(len(model_names)-1)/2)
    axes[0].set_xticklabels([f"{h}m" for h in HORIZONS])
    axes[0].set_ylabel("Test MSE (lower = better)")
    axes[0].set_title("Forecast MSE by horizon", fontweight="bold")
    axes[0].legend(frameon=True, fontsize=7)

    for i, mname in enumerate(model_names):
        r2s = [all_results[h].get(mname, {}).get("avg_r2", np.nan) for h in HORIZONS]
        color = model_colors.get(mname, f"C{i}")
        axes[1].bar(x + i*w, r2s, w, label=mname, color=color, alpha=0.75,
                    edgecolor="white", lw=1)
    axes[1].set_xticks(x + w*(len(model_names)-1)/2)
    axes[1].set_xticklabels([f"{h}m" for h in HORIZONS])
    axes[1].set_ylabel("Test R2"); axes[1].axhline(0, color="grey", lw=1, alpha=0.5)
    axes[1].set_title("Forecast R2 by horizon", fontweight="bold")
    axes[1].legend(frameon=True, fontsize=7)

    fig.tight_layout()
    path1 = f"{output_path}/forecast_regime_comparison.png"
    fig.savefig(path1, dpi=200, bbox_inches="tight"); plt.close(fig)
    print(f"  Saved: {path1}")

    # ── Fig 2: Regime improvement ──
    fig, ax = plt.subplots(figsize=(12, 6))
    fig.suptitle("Regime Awareness: Improvement Over Pooled Ridge",
                 fontweight="bold", fontsize=14, y=1.02)

    regime_models = ["Ridge + regime", "Ridge + regime prob", "Regime-conditional"]
    x = np.arange(len(HORIZONS)); w = 0.8 / len(regime_models)
    for i, mname in enumerate(regime_models):
        deltas = []
        for h in HORIZONS:
            pooled = all_results[h].get("Pooled Ridge", {}).get("avg_mse", np.nan)
            regime = all_results[h].get(mname, {}).get("avg_mse", np.nan)
            delta_pct = (regime - pooled) / pooled * 100 if pooled > 0 else np.nan
            deltas.append(delta_pct)
        color = model_colors.get(mname, f"C{i}")
        ax.bar(x + i*w, deltas, w, label=mname, color=color, alpha=0.75,
               edgecolor="white", lw=1)
    ax.set_xticks(x + w*(len(regime_models)-1)/2)
    ax.set_xticklabels([f"{h}m" for h in HORIZONS])
    ax.axhline(0, color="grey", lw=1.5)
    ax.set_ylabel("% change in MSE vs Pooled Ridge (negative = better)")
    ax.set_title("Does conditioning on regimes reduce forecast error?", fontweight="bold")
    ax.legend(frameon=True, fontsize=9)

    fig.tight_layout()
    path2 = f"{output_path}/forecast_regime_improvement.png"
    fig.savefig(path2, dpi=200, bbox_inches="tight"); plt.close(fig)
    print(f"  Saved: {path2}")

    # ── Fig 3: Subsample stability ──
    if period_results:
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        fig.suptitle("Why 12-Month Inflation Forecasting Fails: Subsample Analysis",
                     fontweight="bold", fontsize=14, y=1.02)

        pnames = [p["period"] for p in period_results]
        r2_ars = [p["r2_ar"] for p in period_results]
        r2_ridges = [p["r2_ridge"] for p in period_results]
        var_pis = [p["var_pi"] for p in period_results]

        ax = axes[0]
        ax.scatter(var_pis, r2_ridges, s=80, color=C1, zorder=3, edgecolors="white", lw=1)
        for p, vp, r2 in zip(pnames, var_pis, r2_ridges):
            ax.annotate(p, (vp, r2), fontsize=8, textcoords="offset points", xytext=(5, 5))
        ax.axhline(0, color="grey", ls="--", lw=1)
        ax.set_xlabel("Inflation variance in test period")
        ax.set_ylabel("12-month forecast R2")
        ax.set_title("Forecast quality vs inflation variability", fontweight="bold")

        ax = axes[1]
        x_p = np.arange(len(pnames))
        ax.bar(x_p - 0.2, r2_ars, 0.35, label="AR", color="#6b7280", alpha=0.75, edgecolor="white")
        ax.bar(x_p + 0.2, r2_ridges, 0.35, label="Ridge", color=C1, alpha=0.75, edgecolor="white")
        ax.set_xticks(x_p); ax.set_xticklabels(pnames, rotation=30, ha="right", fontsize=9)
        ax.axhline(0, color="grey", ls="--", lw=1)
        ax.set_ylabel("12-month forecast R2")
        ax.set_title("R2 by test period", fontweight="bold")
        ax.legend(frameon=True, fontsize=9)

        fig.tight_layout()
        path3 = f"{output_path}/forecast_subsample_stability.png"
        fig.savefig(path3, dpi=200, bbox_inches="tight"); plt.close(fig)
        print(f"  Saved: {path3}")
    else:
        path3 = None

    return [p for p in [path1, path2, path3] if p]


# ── Main ───────────────────────────────────────────────────────────


all_results = run_forecasting(regime_df, regime_names, n_regimes)
period_results = subsample_stability(regime_df)
paths = plot_results(all_results, period_results, OUTPUT_PATH)

print(f"\nOutputs: {paths}")
print("Done.")

# ── Simpson's paradox quantification ─────────────────────────
ac12_full = regime_df["inflation"].autocorr(lag=12)
ac12_within = np.average(
    [regime_df[regime_df["regime"]==r]["inflation"].autocorr(lag=12) for r in range(n_regimes)],
    weights=[len(regime_df[regime_df["regime"]==r]) for r in range(n_regimes)])
comp = (ac12_full - ac12_within) / ac12_full * 100
print(f"\n  Full-sample AC(12)={ac12_full:.3f}, within-regime={ac12_within:.3f}")
print(f"  Compositional share: {comp:.0f}% of apparent persistence is from mixing.")


In [ ]:
"""Summary."""
import glob
print("="*70)
print("SUMMARY")
print("="*70)
print("""
1. REGIMES: Low ~2.4%, High ~7.4%. Stable pre/post 2020.
2. TAYLOR RULES: Specification-dependent. HAC SE 2-3x OLS.
3. BREAKS: None survive sup-Wald with CBO NAIRU.
4. FORECASTING: Random walk competitive. All fail beyond 3m.
5. YIELD CURVE: No regime-transition signal. Recession LORO AUC~0.74.
""")
outputs = sorted(glob.glob(f"{OUTPUT_PATH}/*.png"))
print(f"Outputs ({len(outputs)}):")
for p in outputs: print(f"  {os.path.basename(p)}")
